In [ ]:
import os
import sys
import glob
import fnmatch
import random
os.getcwd()             #Get Current working directory
os.listdir(path)        #Show all files into the current directory

In [ ]:
def Build_File_List (dataset_dir , out_dir , splits_num=4 , shuffle=False):
    
    """
    frames_dir : directory for the frames to be processed -One video frames-
    out_dir    : directory where the code gonna put the list file
    rgb_prefix : frames' prefix (eg:frame0012)
    splits_num : number of dataset spilts 
    shuffle    : True = Shuffle , False = Unshuffled
    """
    Name_Label = SpiltsInfoExtract1()
    Frames_dir = SpiltsInfoExtract2(dataset_dir)
    
    for i in range(splits_num):
        Train_DFL,Test_DFL = MergeInfo(Name_Label,Frames_dir, i, shuffle)
        open(os.path.join(out_dir, 'rgb_train_FileList{}.txt'.format(i+1)), 'w').writelines(Train_DFL)
        open(os.path.join(out_dir, 'rgb_test_FileList{}.txt'.format(i+1)), 'w').writelines(Test_DFL)

In [ ]:
def SpiltsInfoExtract1():
    """
    Extract name&label for each video 
    SplitsInfo [split index-1][trian or test][video index] : the info for the 1st video in 1st train split ('v_ApplyEyeMakeup_g08_c01', 0)
    SplitsInfo : List of tuples -Number of tuples = number of spilts (4)- each tuple has 2 lists, each list has 2 tuples, tuple[0] is a trainlist 
    containing tuples each tuple have the video name and the label for specific video in specific split,tuple[1] is a testlist just like the previous tuple.
    """
    IndClass = [x.strip().split() for x in open('classInd.txt')]  #[['1',classname1],['2',classname2],....]
    ClassIndDic = {x[1]:int(x[0])-1 for x in IndClass}            #{classname1:0 , classname2:1 ,...}
    
    
    
    def ExtractInfo(line):
        """
        Input  : line form testlist or trainlist (eg : ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi )
        Output : the name and the label for this video
        """
        line = line.split('/')
        name = line[1].split('.')[0]
        label = ClassIndDic[line[0]]
        return name,label
    
    Name_Label = []
    
    for i in range(1,5): #looping through the dataset splits to Extract info
        trainlist = [ExtractInfo(x) for x in open ('trainlist{:02d}.txt'.format(i))] #Extract info from every video in the trian splits
        testlist  = [ExtractInfo(x) for x in open ('testlist{:02d}.txt'.format(i))]  #Extract info from every video in the test splits
        Name_Label.append((trainlist,testlist))

    return Name_Label

In [ ]:
def SpiltsInfoExtract2(dataset_dir):
    '''
    Input : dataset directory 
    Output : RGB_count(dictionary) mapping video names inside the class folder with the number of frames inside
             Video_dir (dictionary) mapping video names inside the class folder with the directory for this folder
    '''
    Framefiles_dir = glob.glob(os.path.join(dataset_dir,'*'))   #List contain all directories for frame files inside a class
    
    RGB_count = {}
    Video_dir = {}
    for i,file_dir in enumerate (Framefiles_dir) :
        Video_name = file_dir.split('/')[-1]                  #get video name = folder name
        Frames_list = os.listdir(file_dir)                    #list contain all frame names inside video folders
        RGB_count[Video_name] = len(Frames_list)              #mapping frames num with video names
        Video_dir[Video_name] = file_dir                      #mapping video directories with video names
                         
    return RGB_count,Video_dir

In [ ]:
def MergeInfo(Name_Label,Frames_dir, split_idx, shuffle=False):
    '''
    Name_Label : The output of SpiltsInfoExtract1
    Frames_dir : The Output of SpiltsInfoExtract2
    split_inx : 1 to 4 (split number)
    return Train_DFL : Huge string every line of it consist of [Dirctory of the video -- number of frames -- label]
           Test_DFL  : Huge string every line of it consist of [Dirctory of the video -- number of frames -- label]
    '''
    Name_Label = Name_Label[split_idx]                               #Specify which split being processed (output: tuple(trainlist,testlist)
    train_info =  Name_Label[0]                                      #List of tuples each tuple is (name,label)
    test_info  =  Name_Label[1]                                      #List of tuples each tuple is (name,label)
    
    def DFL (Name_Label):                                            #DFL : Directory , Frames , Label
        RGB_list = []
        for name_label in Name_Label:                                #For each video in the split
            Video_dir = Frames_dir[1][name_label[0]]
            RGB_count = Frames_dir[0][name_label[0]]
            Label     = name_label[1]
            RGB_list.append('{} {} {}\n'.format(Video_dir, RGB_count, Label))     #packing variables into string -huge one-

            if shuffle:
                random.shuffle(RGB_list)
        return RGB_list
    
    Train_DFL = DFL(train_info)
    Test_DFL  = DFL(test_info)
    
    return Train_DFL,Test_DFL